# Price Index of Operating Costs (PIOC)
**Developed by**: Itzamna Huerta, for the Association for Neighborhood and Housing Development (ANHD)  
**Created**: Feb 2025  
**Last Updated**: 03/19/2025


#### Overview
The Price Index of Operating Costs (PIOC) is an annual report published by the New York City Rent Guidelines Board (RGB), tracking changes in the costs of maintaining rent-stabilized housing. This project automates the extraction and analysis of PIOC data to support advocacy efforts, policy analysis, and preservation campaigns.

By systematically collecting and analyzing cost trends, this project provides insight into the financial pressures faced by property owners of rent-stabilized units, informing decisions on affordability policies and tenant protections.

However, due to inconsistencies in the PDF text structure and formatting variations across years, certain extracted values require manual corrections. For example, in the 2023 and 2022 reports, the automated extraction misidentified the percentage change for Administrative Costs, requiring post-processing adjustments. Similarly, some 2019 values (e.g., Maintenance, Fuel, and Labor Costs) were not reliably captured through regex-based searches and had to be manually referenced against the original document. While these corrections ensure accuracy, they also highlight the limitations of text extraction methods when dealing with complex document layouts. Future improvements may involve refining text parsing logic or integrating machine learning-based entity recognition for more precise extraction.
#### Objectives
<u>Extract Key Variables</u>: Identify and capture percentage changes for major cost categories:
- Rent Stabilized Apartments increased
- Real estate taxes
- Insurance Costs
- Adminstrative Costs
- Maintenance 
- Utilities
- Labor Costs
- Fuel 
- Natural Gas & Fuel Oil Heat


<u>Standardize Data</u>: Clean and structure extracted values to ensure consistency across multiple years.

<u>Analyze Cost Trends</u>: Compile data into a structured format for visualization and policy analysis.


#### Methodology
1. Text Extraction & Preprocessing: Extract relevant text from PIOC PDF reports, clean and standardize formatting.
2. Pattern Matching: Use regex-based searches to locate and extract percentage changes associated with key cost categories.
3. Multi-Year Data Compilation: Iterate through multiple reports, associate extracted values with corresponding years, and consolidate into a structured dataset.
4. Data Transformation & Visualization: Convert data into a numerical format, reshape for analysis, and prepare for visualization.

In [2]:
# Import libraries
import pandas as pd
import pdfplumber 
import re
import os

<hr>


## Phase 1: Analyzing Text Structure and Developing Extraction Methodology

The first phase focuses on understanding the structure of PIOC reports and developing an approach to extract percentage values for key cost categories. Since PIOC data is presented in PDF format with varying layouts, this phase involves:


1. Text Cleaning & Preprocessing: Removing unnecessary line breaks and formatting inconsistencies to improve data extraction accuracy.
2. Keyword Identification: Defining key cost categories and their variations to ensure comprehensive data capture.
3. Pattern Recognition & Matching: Using regex-based searches to locate percentage values associated with each category.
4. Validation & Debugging: Testing extraction logic on a single PDF before scaling to multiple years.


In [3]:
# First attempt one pdf file before creating a function to iteration through all pdf files in folder 
pdf = pdfplumber.open('./pdfs/2019-PIOC.pdf')

In [4]:
# Define keywords (with expected variations in phrasing)
keywords = {
    "Rent Stabilized Apartments increased": None,
    "Real estate taxes": None,
    "Insurance costs": None,
    "Administrative Costs": None,
    "Maintenance": None,
    "Utilities": None,
    "Labor costs": None,
    "Fuel": None,
    "Natural gas": None,
}

first_page = pdf.pages[2] # Accessing the first page
text = first_page.extract_text() # Extracting text from page
# print(text) # Uncomment to see the original text from the page

# Preprocess text: Remove extra line breaks to keep sentences intact
clean_text = re.sub(r'\n+', ' ', text)
print("Preproccessed text:\n\n",clean_text)


Preproccessed text:

 New York City Rent Guidelines Board 2019 Price Index of Operating Costs 04 Introduction What’s New 04 Overview R The Price Index of Operating Costs (PIOC) for Rent Stabilized Apartment Buildings increased 5.5% this year. 05 Price Index R Costs in natural-gas heated buildings increased 5.5%, Components while costs in fuel-oil heated buildings increased 5.8%. R The “Core” PIOC, which excludes the changes in fuel oil 08 PIOC by Building prices, natural gas, and steam costs, is useful for analyzing Type inflationary trends. The Core PIOC rose by 4.9% this year. R Fuel costs increased 13.8%, the highest increase of any 08 Rent Stabilized PIOC component. Hotels R Real estate taxes increased 7.1%, primarily due to a rise in assessments for Class Two properties. 09 Rent Stabilized Lofts R The Utilities component increased by 0.4%, primarily due to an increase in water and sewer costs. 09 The Core PIOC R The Price Index of Operating Costs for Rent Stabilized Apartment Buil

In [5]:
# Iterate through keywords and find percentages
for key in keywords.keys():
    # Regex pattern to find the keyword followed by a percentage in the same sentence
    pattern = rf"({re.escape(key)}.*?)(\d{{1,3}}(?:\.\d+)?%)"  # Captures the percentage

    match = re.search(pattern, clean_text, re.IGNORECASE)  # Case-insensitive search

    if match:
        # Store the percentage for the keyword
        keywords[key] = match.group(2)

    # Debugging: Print what’s being matched
    # print(f"Checking: {key} → Match: {match.groups() if match else 'None'}")

# Print final output
print("Final Output:\n\n", keywords)

Final Output:

 {'Rent Stabilized Apartments increased': None, 'Real estate taxes': '7.1%', 'Insurance costs': None, 'Administrative Costs': None, 'Maintenance': None, 'Utilities': '0.4%', 'Labor costs': None, 'Fuel': '5.8%', 'Natural gas': '4.9%'}


## Phase 2: Extracting and Consolidating PIOC Data Across Multiple Reports

This phase focuses on automating the extraction of PIOC percentage data from multiple annual reports and compiling the results into a structured dataset for analysis. The key steps include:

- Batch Processing PDFs: Iterating through multiple PIOC reports to extract percentage values for each cost category.
- Standardizing Data Structure: Associating extracted values with the correct year and ensuring consistency across reports.
- Transforming Data for Analysis: Converting extracted data into a structured DataFrame, enabling easy manipulation and visualization.
- Compiling Historical Trends: Merging data from all reports into a single dataset to track year-over-year cost changes.

By the end of this phase, a comprehensive dataset is created, allowing for deeper analysis of operating cost trends in NYC rent-stabilized buildings.









In [6]:
def extract_pioc_from_pdf(pdf_path, keywords, year):
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ""
        for page_num in range(2, len(pdf.pages)):  # Start from page 3 (index 2)
            page = pdf.pages[page_num]
            text = page.extract_text()
            all_text += text

    # Preprocess text: Remove extra line breaks and unify formatting
    clean_text = re.sub(r'\n+', ' ', all_text)

    # Dictionary to store extracted data
    extracted_data = {'Year': year}

    # Define patterns for increase and decrease terms
    increase_keywords = ["increase", "increased", "rose", "rising"]
    decrease_keywords = ["decrease", "decreased", "falling", "fell", "declined"]

    # Iterate through keywords to find relevant data
    for key in keywords.keys():
        # Regex to find sentences containing the keyword followed by a percentage
        pattern = rf"({re.escape(key)}.*?)(\d{{1,3}}(?:\.\d+)?%)"

        match = re.search(pattern, clean_text, re.IGNORECASE)

        if match:
            percentage = match.group(2)  # Extract percentage
            sentence = match.group(1)  # Extract full sentence containing the keyword

            # Determine if the change is positive or negative
            if any(word in sentence.lower() for word in decrease_keywords):
                percentage = f"-{percentage}"
            elif any(word in sentence.lower() for word in increase_keywords):
                percentage = f"{percentage}"  # Explicitly keep positive

            # Store the extracted percentage
            extracted_data[key] = percentage

    return extracted_data

In [7]:
# Define a list of PDF file paths and years
pdf_paths = [
    './pdfs/2024-PIOC.pdf',
    './pdfs/2023-PIOC.pdf',
    './pdfs/2022-PIOC.pdf',
    './pdfs/2021-PIOC.pdf',
    './pdfs/2020-PIOC.pdf',
    './pdfs/2019-PIOC.pdf',
]

# Define keywords (with expected variations in phrasing)
keywords = {
    "Rent Stabilized Apartments increased": None,
    "Real estate taxes": None,
    "Insurance costs": None,
    "Administrative costs": None,
    "Maintenance": None,
    "Utilities": None,
    "Labor costs": None,
    "Fuel": None,
    "Natural gas": None
}

# Create an empty DataFrame to hold the final result
final_df = pd.DataFrame()

# Iterate through each PDF file and extract the percentages
for i, pdf_path in enumerate(pdf_paths):
    year = 2024 - i  # Assuming 2024 is the starting year, subtract i to get the year
    print(f"Processing: {pdf_path} for year {year}")
    
    # Copy the keyword dictionary to avoid overwriting data
    keywords_data = extract_pioc_from_pdf(pdf_path, keywords.copy(), year)
    
    # Convert the extracted data to a DataFrame and append to the final DataFrame
    df = pd.DataFrame([keywords_data])
    final_df = pd.concat([final_df, df], ignore_index=True)

Processing: ./pdfs/2024-PIOC.pdf for year 2024
Processing: ./pdfs/2023-PIOC.pdf for year 2023
Processing: ./pdfs/2022-PIOC.pdf for year 2022
Processing: ./pdfs/2021-PIOC.pdf for year 2021
Processing: ./pdfs/2020-PIOC.pdf for year 2020
Processing: ./pdfs/2019-PIOC.pdf for year 2019


In [8]:
# Reorder columns to place 'Year' first
final_df = final_df[['Year'] + [col for col in final_df.columns if col != 'Year']]

# Print the final DataFrame
final_df

,Year,Rent Stabilized Apartments increased,Real estate taxes,Insurance costs,Administrative costs,Maintenance,Utilities,Labor costs,Fuel,Natural gas
0,2024,3.9%,3.2%,21.7%,4.6%,3.5%,1.3%,4.3%,-7.1%,4.9%
1,2023,8.1%,7.7%,19.9%,2.9%,9.4%,8.8%,2.9%,19.9%,7.2%
2,2022,4.2%,-3.7%,19.6%,3.7%,9.2%,5.8%,4.1%,4.3%,3.0%
3,2021,3.0%,3.9%,18.8%,-0.7%,3.1%,2.1%,2.8%,1.6%,3.5%
4,2020,3.7%,5.9%,16.5%,3.5%,4.8%,1.6%,3.2%,3.7%,5.1%
5,2019,NaN,7.1%,6.0%,6.0%,7.1%,0.4%,7.1%,5.8%,4.9%


Manually updating the following cells due to time constraints and errors in getting them through the function:

- 2023: Administrative Costs (3.3%)
- 2022: Administrative Costs (6.7%)
- 2019: Administrative Costs (3.5%)

- 2019: Maintenance (3.8%)

- 2019: Labor Costs (6.0%)
- 2019: Fuel (13.8%)



In [9]:
# Manually correct the specific values in final_df
final_df.loc[final_df["Year"] == 2023, "Administrative costs"] = "3.3%"
final_df.loc[final_df["Year"] == 2022, "Administrative costs"] = "6.7%"
final_df.loc[final_df["Year"] == 2019, "Administrative costs"] = "3.5%"

final_df.loc[final_df["Year"] == 2019, "Maintenance"] = "3.8%"
final_df.loc[final_df["Year"] == 2019, "Labor costs"] = "6.0%"
final_df.loc[final_df["Year"] == 2019, "Fuel"] = "13.8%"



# This was previously NaN but have cross referenced the pdf for the right value
final_df.loc[final_df["Year"] == 2019, "Rent Stabilized Apartments increased"] = "5.5%"

In [9]:
# Convert percentages to numeric values
for col in final_df.columns:
    if col != "Year":
        final_df[col] = final_df[col].str.rstrip('%').astype(float)


In [ ]:
# Export to CSV
final_df_melted.to_csv("./data/pioc_2024-2019-un.csv", index=False)

In [11]:
# Reshape DataFrame to long format for easier plotting
final_df_melted = final_df.melt(id_vars=["Year"], var_name="Cost Category", value_name="Percentage Change")

In [12]:
# Display the transformed DataFrame
final_df_melted.head()

,Year,Cost Category,Percentage Change
0,2024,Rent Stabilized Apartments increased,3.9
1,2023,Rent Stabilized Apartments increased,8.1
2,2022,Rent Stabilized Apartments increased,4.2
3,2021,Rent Stabilized Apartments increased,3.0
4,2020,Rent Stabilized Apartments increased,3.7


In [13]:
# Export to CSV
# final_df_melted.to_csv("./data/pioc_2024-2019-.csv", index=False)